In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [ ]:
datasetIncendios = pd.read_csv("./modis_2021_Spain_incendios_with_clima.csv")
datasetIncendios = datasetIncendios.query('latitude > 30')

datasetIncendios["Temperatura máxima (ºC)"] =  np.zeros(len(datasetIncendios))
datasetIncendios["Temperatura mínima (ºC)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Temperatura media (ºC)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Racha (km/h)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Velocidad máxima (km/h)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Precipitación 00-24h (mm)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Precipitación 00-06h (mm)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Precipitación 06-12h (mm)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Precipitación 12-18h (mm)"] = np.zeros(len(datasetIncendios))
datasetIncendios["Precipitación 18-24h (mm)"] = np.zeros(len(datasetIncendios))
datasetIncendios.to_csv("./modis_2021_Spain_incendios_with_clima.csv", index=False)

In [ ]:
columns = ['Temperatura máxima (ºC)',
'Temperatura mínima (ºC)',
'Temperatura media (ºC)',
'Racha (km/h)',
'Velocidad máxima (km/h)',
'Precipitación 00-24h (mm)',
'Precipitación 00-06h (mm)',
'Precipitación 06-12h (mm)',
'Precipitación 12-18h (mm)',
'Precipitación 18-24h (mm)']
datasetIncendios = pd.read_csv("./modis_2021_Spain_incendios_with_clima.csv")

def getClosestValue(index, colum):
    columnIndexdate = datasetIncendios.columns.get_loc("acq_date")
    print(datasetIncendios.iloc[index,columnIndexdate])
    df = pd.read_csv(f'./ClimaDataSmall/Aemet{datasetIncendios.iloc[index,columnIndexdate]}.csv')
    columnIndex = df.columns.get_loc(colum)
    df = df.iloc[:,[-1,-2,columnIndex]]
    df = df.query('Latitude > 30 and Latitude < 44 and Longitude > -20')
    if str(df.iloc[1,2]).find(' ') != -1:
        for _ in range(len(df)):
            df.iloc[_,2] = float(df.iloc[_,2][:df.iloc[_,2].find(' ')])
    a = []
    for _ in df.iloc[:, 2].values:
        a.append(float(_))

    df2 = df.drop([colum], 1)
    df3 = df[colum]
    lat = df2['Latitude']
    lon = df2['Longitude']
    values = np.array(df3,dtype='float64')
    arraylat = np.array(lat,dtype='float64')
    arraylon = np.array(lon,dtype='float64')
    est_u = griddata((arraylat, arraylon), values, (datasetIncendios.iloc[index,0],datasetIncendios.iloc[index,1]), method='linear')
    columnIndex = datasetIncendios.columns.get_loc(colum)
    datasetIncendios.iloc[index,columnIndex] = est_u
    print(f"{colum} : {est_u}" )


for index in range(len(datasetIncendios)):
    print(f"{index} of : {len(datasetIncendios)}")
    for colum in columns:
        getClosestValue(index, colum)

datasetIncendios.to_csv("./modis_2021_Spain_incendios_with_clima2.csv", index=False)

In [7]:

datasetIncendios = pd.read_csv("./modis_2021_Spain_incendios_with_clima2.csv")
datasetIncendios

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,...,Temperatura máxima (ºC),Temperatura mínima (ºC),Temperatura media (ºC),Racha (km/h),Velocidad máxima (km/h),Precipitación 00-24h (mm),Precipitación 00-06h (mm),Precipitación 06-12h (mm),Precipitación 12-18h (mm),Precipitación 18-24h (mm)
0,37.0886,-5.8802,300.4,1.1,1.0,2021-01-02,1125,Terra,MODIS,37,...,12.303982,-0.509447,5.897267,22.482428,14.482428,0.000000,0.0,0.000000,0.000000,0.000000
1,36.6599,-4.8777,305.5,1.8,1.3,2021-01-02,1304,Aqua,MODIS,62,...,12.378087,4.477107,8.447569,32.935466,16.503344,0.034612,0.0,0.000000,0.034612,0.000000
2,36.8729,-2.4486,304.8,1.2,1.1,2021-01-02,2228,Terra,MODIS,62,...,14.526109,7.572479,11.056317,53.756511,37.522342,0.000000,0.0,0.000000,0.000000,0.000000
3,36.8745,-2.4347,334.7,1.2,1.1,2021-01-02,2228,Terra,MODIS,100,...,14.592195,7.830187,11.217722,54.492696,38.231562,0.000000,0.0,0.000000,0.000000,0.000000
4,40.2352,-0.8699,320.1,1.1,1.1,2021-01-04,1112,Terra,MODIS,80,...,7.471121,-2.678443,2.446339,29.316719,14.414542,0.447523,0.0,0.298348,0.000000,0.149174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2903,43.2687,-5.2530,307.4,1.1,1.0,2021-12-31,2206,Terra,MODIS,71,...,16.848519,5.495884,11.166160,18.005546,11.167271,0.000000,0.0,0.000000,0.000000,0.000000
2904,43.1755,-5.4329,309.2,1.1,1.0,2021-12-31,2206,Terra,MODIS,77,...,16.401154,4.121999,10.257632,16.592463,10.603606,0.000000,0.0,0.000000,0.000000,0.000000
2905,43.2442,-3.7813,316.1,1.0,1.0,2021-12-31,2206,Terra,MODIS,93,...,19.817953,7.562652,13.676781,38.696725,23.395654,0.000000,0.0,0.000000,0.000000,0.000000
2906,43.1631,-5.5095,303.0,1.1,1.0,2021-12-31,2206,Terra,MODIS,47,...,16.484505,3.574733,10.023610,15.983789,10.301446,0.000000,0.0,0.000000,0.000000,0.000000
